# Text Data in scikit-learn

In [ ]:
import matplotlib.pyplot as plt
import sklearn

sklearn.set_config(display='diagram')

In [ ]:
from pathlib import Path
import tarfile
from urllib import request

data_path = Path("data")
extracted_path = Path("data") / "train"
imdb_path = data_path / "aclImdbmini.tar.gz"

def untar_imdb():
    if extracted_path.exists():
        print("imdb dataset already extracted")
        return
    with tarfile.open(imdb_path, "r") as tar_f:
        tar_f.extractall(data_path)
            
# This may take some time to run since it will download and extracted
untar_imdb()

## CountVectorizer

In [ ]:
sample_text = ["Can we go to the hill? I finished my homework.",
               "The hill is very tall. Please be careful"]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer()
vect.fit(sample_text)

vect.get_feature_names()

In [ ]:
X = vect.transform(sample_text)
X

In [ ]:
X.toarray()

### Bag of words

In [ ]:
sample_text

In [ ]:
X_inverse = vect.inverse_transform(X)

In [ ]:
X_inverse[0]

In [ ]:
X_inverse[1]

## Loading text data with scikit-learn

In [ ]:
from sklearn.datasets import load_files

reviews_train = load_files(extracted_path, categories=["neg", "pos"])
raw_text_train, raw_y_train = reviews_train.data, reviews_train.target
raw_text_train = [doc.replace(b"<br />", b" ") for doc in raw_text_train]

In [ ]:
import numpy as np
np.unique(raw_y_train)

In [ ]:
np.bincount(raw_y_train)

In [ ]:
len(raw_text_train)

In [ ]:
raw_text_train[5]

## Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

text_train, text_val, y_train, y_val = train_test_split(
    raw_text_train, raw_y_train, stratify=raw_y_train, random_state=0)

### Transform training data

In [ ]:
vect = CountVectorizer()
X_train = vect.fit_transform(text_train)

In [ ]:
len(text_train)

In [ ]:
X_train

### Transform validation set

In [ ]:
len(text_val)

In [ ]:
X_val = vect.transform(text_val)

In [ ]:
X_val

### Extract feature names

In [ ]:
feature_names = vect.get_feature_names()

In [ ]:
feature_names[10000:10020]

In [ ]:
feature_names[::3000]

### Linear model for classification

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear', random_state=42).fit(X_train, y_train)

In [ ]:
lr.score(X_val, y_val)

In [ ]:
def plot_important_features(coef, feature_names, top_n=20, ax=None, rotation=40):
    if ax is None:
        ax = plt.gca()
    feature_names = np.asarray(feature_names)
    inds = np.argsort(coef)
    low = inds[:top_n]
    high = inds[-top_n:]
    important = np.hstack([low, high])
    myrange = range(len(important))
    colors = ['red'] * top_n + ['blue'] * top_n
    
    ax.bar(myrange, coef[important], color=colors)
    ax.set_xticks(myrange)
    ax.set_xticklabels(feature_names[important], rotation=rotation, ha="right")
    ax.set_xlim(-.7, 2 * top_n)
    ax.set_frame_on(False)

In [ ]:
feature_names = vect.get_feature_names()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_important_features(lr.coef_.ravel(), feature_names, top_n=20, ax=ax)

## Exercise 1

1. Train a `sklearn.ensemble.RandomForestClassifier` on the training set, `X_train` and `y_train`.
2. Evalute the accuracy on the validation set.
3. What are the top 20 important features accourind go `feature_importances_` of the random forst.

In [ ]:
# %load solutions/01-ex01-solutions.py

## CountVectorizer Options

In [ ]:
sample_text = ["Can we go to the hill? I finished my homework.",
               "The hill is very tall. Please be careful"]

In [ ]:
vect = CountVectorizer()
vect.fit(sample_text)
vect.get_feature_names()

### Stop words

In [ ]:
vect = CountVectorizer(stop_words='english')
vect.fit(sample_text)
vect.get_feature_names()

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print(list(ENGLISH_STOP_WORDS))

### Max features

In [ ]:
vect = CountVectorizer(max_features=4, stop_words='english')
vect.fit(sample_text)
vect.get_feature_names()

### Min frequency on the imdb dataset

With `min_df=1`

In [ ]:
X_train.shape

`min_df=5`

In [ ]:
vect = CountVectorizer(min_df=4)
X_train_min_df_4 = vect.fit_transform(text_train)

In [ ]:
X_train_min_df_4.shape

In [ ]:
lr_df_4 = LogisticRegression(solver='liblinear', random_state=42).fit(X_train_min_df_4, y_train)

In [ ]:
X_val_min_df_4 = vect.transform(text_val)

#### Scores with different min frequencies

In [ ]:
lr_df_4.score(X_val_min_df_4, y_val)

In [ ]:
lr.score(X_val, y_val)

## Pipelines and Vectorizers

In [ ]:
from sklearn.pipeline import Pipeline

log_reg = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear'))
])

In [ ]:
log_reg

In [ ]:
text_train[:2]

In [ ]:
log_reg.fit(text_train, y_train)

In [ ]:
log_reg.score(text_train, y_train)

In [ ]:
log_reg.score(text_val, y_val)

## Exercise 2

1. Create a pipeline with a `CountVectorizer` with `min_df=5` and `stop_words='english'` and a `RandomForestClassifier`.
2. What is the score of the random forest on the validation dataset?

In [ ]:
# %load solutions/01-ex02-solutions.py

## Bigrams

`CountVectorizer` takes a `ngram_range` parameter

In [ ]:
sample_text

In [ ]:
cv = CountVectorizer(ngram_range=(1, 1)).fit(sample_text)
print("Vocabulary size:", len(cv.vocabulary_))
print("Vocabulary:", cv.get_feature_names())

In [ ]:
cv = CountVectorizer(ngram_range=(2, 2)).fit(sample_text)
print("Vocabulary size:", len(cv.vocabulary_))
print("Vocabulary:")
print(cv.get_feature_names())

In [ ]:
cv = CountVectorizer(ngram_range=(1, 2)).fit(sample_text)
print("Vocabulary size:", len(cv.vocabulary_))
print("Vocabulary:")
print(cv.get_feature_names())

## n-grams with stop words

In [ ]:
cv_n_gram = CountVectorizer(ngram_range=(1, 3), min_df=4, stop_words="english")
cv_n_gram.fit(text_train)

In [ ]:
len(cv_n_gram.vocabulary_)

In [ ]:
print(cv_n_gram.get_feature_names()[::2000])

In [ ]:
pipe_cv_n_gram = Pipeline([
    ('vectorizer', cv_n_gram),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear'))
])

In [ ]:
pipe_cv_n_gram.fit(text_train, y_train)

In [ ]:
pipe_cv_n_gram.score(text_val, y_val)

In [ ]:
feature_names = pipe_cv_n_gram['vectorizer'].get_feature_names()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_important_features(pipe_cv_n_gram['classifier'].coef_.ravel(), feature_names, top_n=20, ax=ax)

## Tf-idf rescaling

In [ ]:
sample_text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidvect = TfidfVectorizer().fit(sample_text)
tfid_trans = tfidvect.transform(sample_text)

In [ ]:
tfid_trans.toarray()

## Train on the imdb dataset

In [ ]:
log_reg_tfid = Pipeline([
   ('vectorizer', TfidfVectorizer(ngram_range=(1, 3), min_df=4,
                                  stop_words="english")),
   ('classifier', LogisticRegression(random_state=42, solver='liblinear'))
])

In [ ]:
log_reg_tfid.fit(text_train, y_train)

In [ ]:
log_reg_tfid.score(text_val, y_val)

## Exercise 3

1. How many samples are there in the training dataset and test dataset?
1. Construct a pipeline with a `TfidfVectorizer` and `LogisticRegression`.
1. Evalute the pipeline on the test set.
1. Plot the feature importances using `plot_important_features`.

In [ ]:
# %load solutions/01-ex03-solutions.py